## Part D. Bike Rentals
### Parte primera
#### 1. **Dataset**




In [ ]:
data = pd.read_csv('../data/hour.csv')
data.info()

In [ ]:
print("Original columns:", data.columns.tolist())
print("Original shape:", data.shape)

In [ ]:
df = data.drop(columns=['instant','dteday','registered','casual','yr'])

In [ ]:
print("After dropping columns:")
print("New shape:", df.shape)
print("Remaining columns:", df.columns.tolist())

Verificando variables

In [ ]:
df.hist(figsize=(10,6))
plt.tight_layout()
plt.show()

Definiendo variables

In [ ]:
categorico = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
numerico = ['temp', 'atemp', 'hum', 'windspeed','cnt']

Matriz correlaciones

In [ ]:
sns.heatmap(df[numerico].corr(),vmax=1.0, vmin=0.0, square=True,annot=True, cmap="Blues")
plt.show()

In [ ]:
entrada_categorico = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
entrada_numerico = ['temp', 'hum', 'windspeed']

In [ ]:
X = df[entrada_categorico+entrada_numerico]
Y = df['cnt']
x_train,x_test,y_train,y_test=train_test_split(X,Y,
                                               random_state=159,
                                               test_size=0.2)

# Procesamiento
modelos = {
    "Lineal": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso()
}

for nombre, modelo in modelos.items():
    modelo.fit(x_train, y_train)
    train_pred = modelo.predict(x_train)
    test_pred = modelo.predict(x_test)

    print(f"Modelo {nombre}")
    print(f" - MSE entrenamiento: {mean_squared_error(y_train, train_pred):.2f}")
    print(f" - MSE prueba:        {mean_squared_error(y_test, test_pred):.2f}")
    print("-" * 40)

In [ ]:
print("X shape after feature selection:", X.shape)
print("X columns:", X.columns.tolist())
print("Y shape:", Y.shape)
print("Features:", X.columns.tolist())
print("Target: cnt (bike rentals)")

In [ ]:
print("\nData quality check:")
print("X dtypes:")
print(X.dtypes)
print("\nAny missing values in X:")
print(X.isnull().sum())
print("\nAny missing values in Y:")
print(Y.isnull().sum())


In [ ]:
# One hot enconding
df_cat=pd.get_dummies(df[entrada_categorico].astype('object'),
               drop_first=True).astype('int')

# Uniendo datos de entrada
df_entrada=pd.concat([df_cat,df[entrada_numerico]],axis=1)

# Procesamiento
X=df_entrada
Y=df['cnt']
x_train,x_test,y_train,y_test=train_test_split(X,Y,
                                               random_state=159,
                                               test_size=0.2)

modelos = {
    "Lineal": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso()
}

for nombre, modelo in modelos.items():
    modelo.fit(x_train, y_train)
    train_pred = modelo.predict(x_train)
    test_pred = modelo.predict(x_test)

    print(f"Modelo {nombre}")
    print(f" - MSE entrenamiento: {mean_squared_error(y_train, train_pred):.2f}")
    print(f" - MSE prueba:        {mean_squared_error(y_test, test_pred):.2f}")
    print("-" * 40)

usando la segunda parte del codigo

In [ ]:
# One hot enconding
df_cat_dm=pd.get_dummies(df[entrada_categorico].astype('object'),
               drop_first=True).astype('int')

# Uniendo datos de entrada
df_entrada=pd.concat([df_cat_dm, data[entrada_numerico]],axis=1)

  * Predict daily rentals (`cnt`)



In [ ]:
# Procesamiento
X=df_entrada
Y=df['cnt']
x_train,x_test,y_train,y_test=train_test_split(X,Y,
                                               random_state=159,
                                               test_size=0.2)

In [ ]:
print("\nAfter train_test_split:")
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("x_train columns:", x_train.columns.tolist())

   * unit variance



In [ ]:
# Standardize features
print("\nBefore standardization:")
print("x_train shape:", x_train.shape)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print("\nAfter standardization:")
print("x_train_scaled shape:", x_train_scaled.shape)
print("x_test_scaled shape:", x_test_scaled.shape)
print("Mean: ", x_train_scaled.mean(axis=0))
print("Variance: ", x_train_scaled.std(axis=0))

#### 2. **Closed-form OLS**



In [ ]:
# (The mathematical derivation is the same as in Part A)

In [ ]:
# Closed-form OLS
x_train_b = np.c_[np.ones((x_train_scaled.shape[0], 1)), x_train_scaled]  # adding intercept term

beta_hat = np.linalg.inv(x_train_b.T.dot(x_train_b)).dot(x_train_b.T).dot(y_train)

print("Design matrix shape:", x_train_b.shape)

   * Report coefficients and intercept.

In [ ]:
# Report coefficients and intercept
print("Coefficients:", beta_hat[1:])
print("Intercept:", beta_hat[0])

In [ ]:
# Plot predicted vs true on test set
x_test_b = np.c_[np.ones((x_test_scaled.shape[0], 1)), x_test_scaled]  # adding intercept term
y_pred = x_test_b.dot(beta_hat)

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("True Bike Rentals")
plt.ylabel("Predicted Bike Rentals")
plt.title("OLS: Predicted vs True Bike Rentals")
plt.show()

### 3. **Gradient Descent**

 * Implement gradient descent to minimize mean squared error.

In [ ]:
# (The same functions defined in Part A)

In [ ]:
print("=== EXPERIMENT 1: Learning Rate = 0.1 ===")

threshold = 1e-3
step_size = 0.1
max_iter = 10000
theta, theta_prev = np.array([2,1] + [0]*(x_train_b.shape[1]-2)), np.ones(x_train_b.shape[1])
cost_history1 = []
iter = 0

while np.linalg.norm(theta - theta_prev) > threshold and iter < max_iter:
    if iter % 100 == 0:
        current_mse = mse(theta, x_train_b, y_train)
        print('Iteration %d. MSE: %.6f' % (iter, current_mse))

    # Saving the cost for plotting
    cost_history1.append(mse(theta, x_train_b, y_train))

    theta_prev = theta.copy()
    gradient = mse_gradient(theta, x_train_b, y_train)
    theta = theta_prev - step_size * gradient
    iter += 1

# Adding last cost
final_cost1 = mse(theta, x_train_b, y_train)
cost_history1.append(final_cost1)
print('Final iteration %d. MSE: %.6f' % (iter, final_cost1))

# Saving results
theta_gd1 = theta.copy()
iterations1 = iter

print("Converged in %d iterations" % iterations1)
print("Intercept:", theta_gd1[0])
print("Coefficients:", theta_gd1[1:])
print()


In [ ]:
print("=== EXPERIMENT 2: Learning Rate = 0.5 ===")

threshold = 1e-3
step_size = 0.5
max_iter = 10000
theta, theta_prev = np.array([2,1] + [0]*(x_train_b.shape[1]-2)), np.ones(x_train_b.shape[1])
cost_history2 = []
iter = 0

while np.linalg.norm(theta - theta_prev) > threshold and iter < max_iter:
    if iter % 100 == 0:
        current_mse = mse(theta, x_train_b, y_train)
        print('Iteration %d. MSE: %.6f' % (iter, current_mse))

    # Saving the cost for plotting
    cost_history2.append(mse(theta, x_train_b, y_train))

    theta_prev = theta.copy()
    gradient = mse_gradient(theta, x_train_b, y_train)
    theta = theta_prev - step_size * gradient
    iter += 1

# Adding last cost
final_cost2 = mse(theta, x_train_b, y_train)
cost_history2.append(final_cost2)
print('Final iteration %d. MSE: %.6f' % (iter, final_cost2))

# Saving results
theta_gd2 = theta.copy()
iterations2 = iter

print("Converged in %d iterations" % iterations2)
print("Intercept:", theta_gd2[0])
print("Coefficients:", theta_gd2[1:])
print()

In [ ]:
# Showing cost vs. iteration curves
plt.figure(figsize=(10, 6))
plt.plot(cost_history1, label='Learning Rate = 0.1', linewidth=2, color='blue')
plt.plot(cost_history2, label='Learning Rate = 0.5', linewidth=2, color='orange')
plt.xlabel('Iteration')
plt.ylabel('Cost (MSE)')
plt.title('Cost vs Iteration Curves: Comparison - Bike Sharing Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

   * Compare parameters and test error to the closed-form OLS.

In [ ]:
# COMPARE PARAMETERS WITH CLOSED-FORM OLS

print("=== COMPARISON WITH CLOSED-FORM OLS ===")

print("Parameters comparison:")
print("Closed-form OLS:")
print("  Intercept: %.6f" % beta_hat[0])
print("  Coefficients: [", end="")
for i, coef in enumerate(beta_hat[1:]):
    print("%.6f" % coef, end="")
    if i < len(beta_hat[1:]) - 1:
        print(", ", end="")
print("]")

print("\nGradient Descent (Learning Rate = 0.1):")
print("  Intercept: %.6f" % theta_gd1[0])
print("  Coefficients: [", end="")
for i, coef in enumerate(theta_gd1[1:]):
    print("%.6f" % coef, end="")
    if i < len(theta_gd1[1:]) - 1:
        print(", ", end="")
print("]")

print("\nGradient Descent (Learning Rate = 0.5):")
print("  Intercept: %.6f" % theta_gd2[0])
print("  Coefficients: [", end="")
for i, coef in enumerate(theta_gd2[1:]):
    print("%.6f" % coef, end="")
    if i < len(theta_gd2[1:]) - 1:
        print(", ", end="")
print("]")

# Parameter differences
diff1 = np.linalg.norm(theta_gd1 - beta_hat)
diff2 = np.linalg.norm(theta_gd2 - beta_hat)
print("\nParameter differences (L2 norm):")
print("  |θ_GD1 - θ_OLS|: %.8f" % diff1)
print("  |θ_GD2 - θ_OLS|: %.8f" % diff2)

# Test error comparison
test_mse_ols = mse(beta_hat, x_test_b, y_test)
test_mse_gd1 = mse(theta_gd1, x_test_b, y_test)
test_mse_gd2 = mse(theta_gd2, x_test_b, y_test)

print("\nTest Error (MSE):")
print("  Closed-form OLS: %.6f" % test_mse_ols)
print("  Gradient Descent (lr=0.1): %.6f" % test_mse_gd1)
print("  Gradient Descent (lr=0.5): %.6f" % test_mse_gd2)

print("\nTest Error differences:")
print("  |MSE_GD1 - MSE_OLS|: %.8f" % abs(test_mse_gd1 - test_mse_ols))
print("  |MSE_GD2 - MSE_OLS|: %.8f" % abs(test_mse_gd2 - test_mse_ols))

In [ ]:
print("\n=== SEASONAL EFFECTS INVESTIGATION ===")

# With one-hot encoding, we have many more features
feature_names = x_train.columns.tolist()
coefficients = beta_hat[1:]

print("Debug - Final check:")
print(f"Number of feature names: {len(feature_names)}")
print(f"Number of coefficients: {len(coefficients)}")

# Create coefficient analysis
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
}).sort_values('Abs_Coefficient', ascending=False)

print("\nTop 10 Most Important Features (by coefficient magnitude):")
for _, row in coef_df.head(10).iterrows():
    print(f"  {row['Feature']:<15}: {row['Coefficient']:>8.4f}")


In [ ]:
# Visualize seasonal effects
plt.figure(figsize=(15, 10))
# Average rentals by raw season data
plt.subplot(2, 3, 6)
seasonal_avg = df.groupby('season')['cnt'].mean()
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
plt.bar(seasons, seasonal_avg.values)
plt.xlabel('Season')
plt.ylabel('Average Bike Rentals')
plt.title('Average Rentals by Season (Raw Data)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Se deduce de la data que el invierno concentra la menor cantidad de alquileres, mientras que en verano se alcanza el punto máximo. Primavera y otoño muestran valores intermedios, pero aún altos, lo que confirma que el clima es un factor determinante en la demanda del servicio.

In [ ]:
# Visualize seasonal effects
plt.figure(figsize=(15, 10))
# Seasonal coefficients specifically
plt.subplot(2, 3, 2)
seasonal_features = coef_df[coef_df['Feature'].str.contains('season_')]
if len(seasonal_features) > 0:
    plt.bar(['Winter (base)', 'Spring', 'Summer', 'Fall'],
            [0] + seasonal_features['Coefficient'].tolist())
    plt.xlabel('Season')
    plt.ylabel('Coefficient Value')
    plt.title('Season Coefficients (vs Winter baseline)')
    plt.xticks(rotation=45)
else:
    # Fallback: show original seasonal averages
    seasonal_avg = df.groupby('season')['cnt'].mean()
    seasons = ['Winter', 'Spring', 'Summer', 'Fall']
    plt.bar(seasons, seasonal_avg.values)
    plt.xlabel('Season')
    plt.ylabel('Average Bike Rentals')
    plt.title('Average Rentals by Season')
plt.grid(True, alpha=0.3)

**conclusión**

El invierno actúa como categoría base. En comparación, la primavera muestra el mayor incremento esperado, con alrededor de 28 unidades más que el invierno. El verano refleja un aumento de unas 16 unidades y el otoño de unas 10. Todas las estaciones elevan la demanda respecto al invierno, sie

In [ ]:
# Visualize seasonal effects
plt.figure(figsize=(15, 10))
# Weather coefficients
plt.subplot(2, 3, 4)
weather_features = coef_df[coef_df['Feature'].str.contains('weathersit_')]
if len(weather_features) > 0:
    plt.bar(['Clear (base)', 'Misty', 'Light Rain/Snow', 'Heavy Rain'][:len(weather_features)+1],
            [0] + weather_features['Coefficient'].tolist())
    plt.xlabel('Weather Situation')
    plt.ylabel('Coefficient (vs Clear baseline)')
    plt.title('Weather Coefficients')
    plt.xticks(rotation=45)
else:
    # Fallback: show original weather averages
    weather_avg = df.groupby('weathersit')['cnt'].mean()
    weather_labels = ['Clear', 'Misty', 'Light Rain/Snow', 'Heavy Rain'][:len(weather_avg)]
    plt.bar(weather_labels, weather_avg.values)
    plt.xlabel('Weather Situation')
    plt.ylabel('Average Bike Rentals')
    plt.title('Average Rentals by Weather')
    plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

**conclusión:**

El clima despejado es la base. En comparación, la neblina reduce en promedio 16 unidades en cnt, la lluvia ligera o nieve cerca de 2, y la lluvia fuerte menos de 1. En conclusión, todas las condiciones climáticas disminuyen la demanda.


In [ ]:
# Visualize seasonal effects
plt.figure(figsize=(15, 10))
# Temperature, humidity, windspeed
plt.subplot(2, 3, 5)
numeric_features = coef_df[coef_df['Feature'].isin(['temp', 'hum', 'windspeed'])]
plt.bar(numeric_features['Feature'], numeric_features['Coefficient'])
plt.xlabel('Numeric Features')
plt.ylabel('Coefficient Value')
plt.title('Numeric Feature Coefficients')
plt.grid(True, alpha=0.3)

**conclusión de lo aprendido:**

En las variables numéricas, un aumento de una unidad en la temperatura (escala normalizada) se asocia con un incremento de aproximadamente 57 unidades en cnt (número total de bicicletas alquiladas), lo que confirma que temperaturas más altas impulsan la demanda. En contraste, la humedad reduce en promedio 22 unidades y la velocidad del viento alrededor de 6. En conjunto, la temperatura es el factor con mayor efecto positivo, mientras que la humedad y el viento influyen de forma negativa, siendo la humedad el factor más determinante en la caída de la demanda.

El modelo, mediante one-hot encoding, permitió identificar los efectos individuales de cada categoría sobre cnt (número total de bicicletas alquiladas). Se evidenció que la temperatura impulsa fuertemente la demanda, mientras que el clima adverso la reduce. 

## Part D - Linear Models
4. **Baseline**

   * Use `LinearRegression` and confirm the coefficients match your OLS implementation.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(x_train_scaled, y_train)


coef_df = pd.DataFrame({
    "Feature": df_entrada.columns,              # nombres de las variables
    "OLS_from_scratch": beta_hat[1:],           # coeficientes OLS (sin intercepto)
    "LinearRegression_sklearn": lin_reg.coef_   # coeficientes sklearn
})


intercept_df = pd.DataFrame({
    "Feature": ["Intercept"],
    "OLS_from_scratch": [beta_hat[0]],
    "LinearRegression_sklearn": [lin_reg.intercept_]
})

coef_table = pd.concat([intercept_df, coef_df], ignore_index=True)
print(coef_table)


   * Compute $R^2$ and mean squared error on the test set.

In [ ]:
y_pred_test_sklearn = lin_reg.predict(x_test_scaled)

print("MSE (test):", mean_squared_error(y_test, y_pred_test_sklearn))
print("R²  (test):", r2_score(y_test, y_pred_test_sklearn))


### Parte D - Part C. Regularization

5. **Ridge versus Lasso**


In [ ]:
lambdas=np.logspace(-3,2,)
ridge_coefs=[]
lasso_coefs=[]

for l in lambdas:
    ridge_model=Ridge(alpha=l)
    ridge_model.fit(x_train,y_train)
    ridge_coefs.append(ridge_model.coef_)

    lasso_model=Lasso(alpha=l)
    lasso_model.fit(x_train,y_train)
    lasso_coefs.append(lasso_model.coef_)

In [ ]:
plt.figure(figsize=(14,5))

# --- Lasso ---
plt.subplot(1,2,1)
plt.plot(lambdas, lasso_coefs)
plt.xscale('log')
plt.title('Lasso: Coeficientes vs λ')
plt.ylabel('Magnitud')
plt.xlabel('Parámetro de regularización λ (escala log)')
plt.legend(df_entrada.columns, loc='upper right', fontsize=8)
plt.grid(True, ls='--', alpha=0.5)

# --- Ridge ---
plt.subplot(1,2,2)
plt.plot(lambdas, ridge_coefs)
plt.xscale('log')
plt.title('Ridge: Coeficientes vs λ')
plt.xlabel('Parámetro de regularización λ (escala log)')
plt.ylabel('Magnitud')
plt.legend(df_entrada.columns, loc='upper right', fontsize=8)
plt.grid(True, ls='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Kfolds to select the best alpha

ridge_model=Ridge()
lasso_model=Lasso()

K = 5
param_grid = {'alpha': lambdas}
kfold = KFold(K, random_state=0, shuffle=True)

# Modelo Ridge
grid_r = GridSearchCV(ridge_model,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
grid_r.fit(x_train, y_train)

# Modelo Lasso
grid_l = GridSearchCV(lasso_model,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
grid_l.fit(x_train, y_train)

print(f'''
Mejores alphas con Ridge y Lasso
alpha_lasso:{grid_l.best_params_['alpha']}
alpha_ridge:{grid_r.best_params_['alpha']}
''')

In [ ]:
lasso_cv=LassoCV(cv=kfold,
                 alphas=lambdas)
lasso_cv.fit(x_train,y_train)

ridge_cv=RidgeCV(cv=kfold,
                 alphas=lambdas)
ridge_cv.fit(x_train,y_train)

print(f'''
Mejores alphas con Ridge_CV y Lasso_CV
alpha_lasso:{lasso_cv.alpha_}
alpha_ridge:{ridge_cv.alpha_}
''')

In [ ]:
mse_lassocv=((y_test-lasso_cv.predict(x_test))**2).mean()
mse_ridgecv=((y_test-ridge_cv.predict(x_test))**2).mean()

r2_ridge=r2_score(y_test,ridge_cv.predict(x_test))
r2_lasso=r2_score(y_test,lasso_cv.predict(x_test))

print(f'''
Comparación de mse en los datos de prueba
mse_lasso_cv: {mse_lassocv:.6f}
mse_ridge_cv: {mse_ridgecv:.6f}
''')
print(f'''
Comparación de r2 en los datos de prueba
rr2_ridge_cv: {r2_ridge:.6f}
rr2_lasso_cv: {r2_lasso:.6f}
''')

**Engineering & Multicollinearity**

  * Add polynomial features (degree 2) using `PolynomialFeatures`.
  * Re-run Ridge/Lasso and discuss how regularization copes with the enlarged feature space.

In [ ]:
# Todas las combinaciones posibles exceptuando las variables dummie
polinomios=PolynomialFeatures(degree=2,
                              include_bias=False)
X_num_poly=polinomios.fit_transform(df[entrada_numerico])

X_num_poly=pd.DataFrame(X_num_poly)
X_num_poly.columns = X_num_poly.columns.astype(str)

In [ ]:
import warnings
X=pd.concat([X_num_poly,df_cat_dm],axis=1)
Y=df['cnt']

x_train,x_test,y_train,y_test=train_test_split(X,Y,
                                               random_state=159,
                                               test_size=0.2)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    lasso_cv_poly=LassoCV(cv=kfold,
                 alphas=lambdas)
    lasso_cv_poly.fit(x_train,y_train)

    ridge_cv_poly=RidgeCV(cv=kfold,
                 alphas=lambdas)
    ridge_cv_poly.fit(x_train,y_train)

mse_lasso_poly= ((y_test - lasso_cv_poly.predict(x_test))**2).mean()
mse_ridge_poly= ((y_test - ridge_cv_poly.predict(x_test))**2).mean()

print(f'''
Funcion polinomica: lasso vs ridge
mse lasso: {mse_lasso_poly}
mse ridge: {mse_ridge_poly}
''')